# Week 5 - Natural Language Processing (NLP)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
import src.week5_func as wk5
import os
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
# data processed and saved with week5_func.save_as_pkl()
df_raw = pd.read_pickle('./data/df_raw.pkl')
df_raw_test = pd.read_pickle('./data/df_raw_test.pkl')

In [3]:
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

In [4]:
# lets start by predicting just positive stuff.
df_pos = df_raw[df_raw['positive']==1]
str_all=[]
def func(text):
    str_all.append(text)
df_pos.apply(lambda x: func(x['text']),axis=1)

# learn on the first 10 smaples
strrring = ''
for i in range(10):
    strrring+=str_all[i]

print(strrring)
    

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to 

In [5]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(strrring))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in strrring])

In [6]:
(encoded.shape)
int2char

{0: 'Y',
 1: 'I',
 2: 'x',
 3: 'J',
 4: 's',
 5: 'H',
 6: 'h',
 7: 'o',
 8: 'b',
 9: 'a',
 10: 'n',
 11: '<',
 12: 'w',
 13: 'O',
 14: '6',
 15: 'm',
 16: '5',
 17: 'D',
 18: 'v',
 19: "'",
 20: 't',
 21: 'y',
 22: 'B',
 23: ' ',
 24: '\x85',
 25: '0',
 26: 'l',
 27: 'c',
 28: 'A',
 29: 'j',
 30: 'z',
 31: 'r',
 32: '2',
 33: '.',
 34: 'C',
 35: ';',
 36: '7',
 37: '1',
 38: '~',
 39: 'p',
 40: 'i',
 41: 'u',
 42: '!',
 43: 'U',
 44: ')',
 45: '-',
 46: 'F',
 47: 'K',
 48: '*',
 49: 'R',
 50: 'g',
 51: 'G',
 52: 'f',
 53: 'P',
 54: 'L',
 55: 'k',
 56: 'T',
 57: ',',
 58: ':',
 59: 'V',
 60: '8',
 61: 'd',
 62: 'E',
 63: 'M',
 64: '?',
 65: '"',
 66: 'q',
 67: '/',
 68: 'e',
 69: 'W',
 70: 'N',
 71: '3',
 72: '(',
 73: '>',
 74: 'S'}

In [7]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros([len(arr), n_labels], dtype=np.float32)
    
    # Fill the appropriate elements with ones
#     one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
#     print(arr.flatten().tolist())
    one_hot[np.arange(one_hot.shape[0]), arr.flatten().int()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = torch.from_numpy(one_hot.reshape((*arr.shape, n_labels)))
    
    return one_hot
# print(type(line_tensor))

In [15]:
# arr = one_hot_encode(encoded, len(chars))
# np.shape(arr)

In [9]:
def get_batches(arr, seq_length): 
    X=[]
    y=[]
    for ind in range(np.shape(arr)[0]-seq_length):
        X.append(arr[ind:ind+seq_length]) #,:])
        y.append(arr[ind+seq_length])# ,:])
#     return np.moveaxis(np.array(X),0,1),np.array(y)
    return torch.from_numpy(np.array(X).reshape(-1,seq_length)).float(), torch.from_numpy(np.array(y)).float()
X,y = get_batches(encoded,10) #try without onehot.
# y_cold = y.max(1)[1]
print(np.shape(X),np.shape(y))

torch.Size([13606, 10]) torch.Size([13606])


In [13]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size) #(129,128)
        self.i2o = nn.Linear(input_size + hidden_size, output_size) #(129,75)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, inputs, hidden):
#         print('forward sizes',input.size(),hidden.size())
        combined = torch.cat((inputs,hidden),1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

# n_hidden = 128
# n_letters = 1 #len(chars)*10
# n_categories = len(chars)
# rnn = RNN(n_letters, n_hidden, n_categories)

# hidden =torch.zeros(1, n_hidden)
# output, next_hidden = rnn.forward(X[0,0].reshape(1,1), hidden)
# print(output,next_hidden)

In [16]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return int2char[category_i], category_i

# print(categoryFromOutput(output))

In [18]:
print(X.size(),y.size())

torch.Size([13606, 10]) torch.Size([13606])


In [19]:
import random

def randomChoice(l, t):
    randind = random.randint(0,np.shape(l)[0]-1)
    return l[randind ,:], t[randind] #, tc[randind]
def randomTrainingExample():
    randX, randy = randomChoice(X,y) #,y_cold)
#     print(randX)
#     reshaped_str = np.reshape(randX,(10,-1),order='C')
    trainstr = ''
    train = [int2char[ii.item()] for ii in randX]
    for ii in train: trainstr+=ii
#     print('[',trainstr,'][',int2char[randy.item()],']')
    return int2char[randy.item()], trainstr, randy, randX
#     return trainstr, int2char[np.argmax(randy).item()], randX.reshape(1,-1), randy, randy_c
randomTrainingExample()

('e',
 ' the homel',
 tensor(68.),
 tensor([23., 20.,  6., 68., 23.,  6.,  7., 15., 68., 26.]))

In [20]:
criterion = nn.NLLLoss()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters(), lr=0.001)

In [23]:
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    rnn.zero_grad()
#     print(np.shape(line_tensor))
    outputs = []
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i].reshape(1,-1), hidden)
        outputs.append(output)
    outputs = torch.stack(outputs)
    
    loss = criterion(output, category_tensor.reshape(1,).long()) #just use the final output to calculate the loss
#     print('lossis', output, category_tensor)
#     print(torch.min(torch.max(outputs,1)[1].reshape(-1,1).float()),torch.min(torch.max(category_tensor, 1)[1]))
#     loss = criterion(torch.max(outputs,1)[1].reshape(-1,1).float(), category_tensor)
    loss.backward(retain_graph=True)

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(learning_rate, p.grad.data)
#         print('post zero', p.grad.data)
    return output, loss.item()

In [24]:
import time
import math

n_iters = 5000
print_every = 100
plot_every = 100
learning_rate = 0.005

# initiate model
n_hidden = 32
n_letters = 75 #len(chars)*10
n_categories = len(chars)
rnn = RNN(n_letters, n_hidden, n_categories)

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()
# output, loss = train(y_cold, X)
# print(timeSince(start))
for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    line_tensor = one_hot_encode(line_tensor, n_categories)
#     print(line_tensor)
    output, loss = train(category_tensor.reshape(1,1), line_tensor)
#     output, loss = train(y_cold, X)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

100 2% (0m 0s) 4.2649 something  / U ✗ (v)
200 4% (0m 0s) 4.3668 ed by a nu / q ✗ (r)
300 6% (0m 0s) 4.4394 Toni Colle / 5 ✗ (t)
400 8% (0m 1s) 4.4142 But this i / ) ✗ (s)
500 10% (0m 1s) 4.3919 am's chara / ' ✗ (c)
600 12% (0m 1s) 4.4615 seem, init / q ✗ (i)
700 14% (0m 2s) 4.5153  is among  / : ✗ (h)
800 16% (0m 2s) 4.5352 the desper / : ✗ (a)
900 18% (0m 2s) nan s dying fr / S ✗ (o)
1000 20% (0m 2s) nan nts to bel / S ✗ (i)
1100 22% (0m 3s) nan avails of  / S ✗ (h)
1200 24% (0m 3s) nan treets. Th / S ✗ (e)
1300 26% (0m 3s) nan ally dange / S ✗ (r)
1400 28% (0m 4s) nan be obvious / S ✗ (,)
1500 30% (0m 4s) nan  is easily / S ✗ ( )
1600 32% (0m 4s) nan e Pepto by / S ✗ ( )
1700 34% (0m 5s) nan  reality)  / S ✗ (a)
1800 36% (0m 5s) nan ow-key per / S ✗ (f)
1900 38% (0m 5s) nan .... at .. / S ✗ (.)
2000 40% (0m 5s) nan  show, is  / S ✗ (f)
2100 42% (0m 6s) nan ECTOR: I'm / S ✗ ( )
2200 44% (0m 6s) nan  streets.  / S ✗ (T)
2300 46% (0m 6s) nan ntil decid / S ✗ (i)
2400 48% (0m 7s) nan  o

## Try LSTM

In [25]:
train

<function __main__.train(category_tensor, line_tensor)>

In [34]:
class LSTM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
#         embeds = self.word_embeddings(sentence)
#         print(sentence.size())
        lstm_out, self.hidden = self.lstm(sentence, self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
    
def train(category_tensor, line_tensor):
    hidden = lstm.init_hidden()
    lstm.zero_grad()
#     print(line_tensor.size())
#     outputs = []
    for i in range(line_tensor.size()[0]): # 10 chars per input
        output = lstm(line_tensor[i].view(1,1,-1))
#         outputs.append(output)
#     outputs = torch.stack(outputs)
    
    loss = criterion(output, category_tensor.reshape(1,).long()) #just use the final output to calculate the loss
#     print('lossis', output, category_tensor)
#     print(torch.min(torch.max(outputs,1)[1].reshape(-1,1).float()),torch.min(torch.max(category_tensor, 1)[1]))
#     loss = criterion(torch.max(outputs,1)[1].reshape(-1,1).float(), category_tensor)
    loss.backward(retain_graph=True)
    optimizer.step()

    # Add parameters' gradients to their values, multiplied by learning rate
#     for p in lstm.parameters():
#         p.data.add_(learning_rate, p.grad.data)
#         print('post zero', p.grad.data)
    return output, loss.item()

In [ ]:
EMBEDDING_DIM = 75
HIDDEN_DIM = 128
lstm = LSTM(EMBEDDING_DIM, HIDDEN_DIM, len(chars), len(chars))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm.parameters(), lr=0.1)
start = time.time()
n_iters = 5000
print_every = 100
plot_every = 100
for iter in range(1, n_iters + 1):
    lstm.zero_grad()
    category, line, category_tensor, line_tensor = randomTrainingExample()
    line_tensor = one_hot_encode(line_tensor, n_categories)
#     print(line_tensor.size())
    output, loss = train(category_tensor.reshape(1,1), line_tensor)
#     output, loss = train(y_cold, X)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

100 2% (0m 33s) 2.9363 ng her hom /   ✗ (e)
200 4% (1m 57s) 4.0821 />Fine per / i ✗ (f)


In [477]:

class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [19]:
import string

all_letters = string.ascii_letters + " .,;?!1234567890'"

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

seq_len = 10 #read 10 chars at a time
n_hidden = 128 #hidden layer
n_letters = len(all_letters)
n_categories = n_categories
rnn = RNN(n_letters, n_hidden, n_categories)

input = lineToTensor('Albertddfw')
hidden = torch.zeros(1, n_hidden)
output, next_hidden = rnn(input[0], hidden)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


### Generate text

Once the model is trained, we can use it to generate completely new text in the style of your training data.  **Train a model using your original choice of corpus below, and generate some sample sentences.** Don't worry too much about your loss / accuracy during training, but instead check on the text your model is generating. Your generated text should be somewhat coherent, i.e. similar to your training text in structure, and not excessively mispelled.

An **example model architecture** is as follows (feel free to play around with different structures):
* Embedding (for each character in your vocab) of dimension 64
* Dropout of 20% for the embedding input to the RNN
* 2 LSTM layers, each of dimension 512 (play around with the number and dimension of hidden layers)
* Dropout of 50% for each LSTM layer
* Dense softmax layer of same dimension as your vocab size (e.g. if your vocab size is 100, this layer is the probabilty that your output is one of 100 possible characters)
    
**You should understand what each of the above elements are and how they work at a high level by the end of this week's exercise.**

### Generalizing the exercise
How do you think you can apply what you learned in the above exercise to other problems involving text? For example, how would you tackle the previous IMDB sentiment classification task using an RNN architecture? **Discuss below.**

(*Bonus*: create an RNN model for the IMDB classification task and discuss your results. How does the performance compare to your bag of words model?)

# Chapter 4: RNNs from scratch
Now that you understand how to use RNNs, it's time to build a basic one from scratch.  You won't understand how they work until you get stuck in the weeds! 

### Generate text (Version 2)
Your task is now to **build the forward pass of a simple RNN, without using any existing RNN APIs**. You can use PyTorch or Tensorflow (Keras is too high level for this exercise), both of which will automatically handle backpropagation for you.  If you use Tensorflow, please research and use Eager execution - it replaces Tensorflow's default graph / session framework, which is very difficult to learn and debug.

Similar to last week's exercise, create a class for your network (write forward and loss steps, allowing PyTorch or Tensorflow to handle backpropagation for you).  Consider appropriate sizes for your input, hidden and output layers - your __init__ method should take in the params `hidden_size`, `vocab_size`, and `embedding_size` (if you use embeddings). Using these variables, you should initialise three weight layers `input_layer`, `hidden_layer`, and `output_layer`.  In an RNN, you will also have to deal with another item - the `hidden_state`. (Note: your RNN structure may vary slightly from this depending on your learning materials, but the key part is always `hidden_state`)

You should **train your RNN on the same data and task as in Chapter 3.**

**How do the results of your basic RNN compare to your model in Chapter 3?**  What do you think explains the difference in performance? Discuss below.

Some relevant resources on LSTMs (and RNN theory) below if you are interested:
* http://colah.github.io/posts/2015-08-Understanding-LSTMs/
* https://www.youtube.com/watch?v=93rzMHtYT_0&list=LLpNVCNE9cYqVrjb2O8bZUGg&index=2&t=0s
* https://www.youtube.com/watch?v=zQxm3Upr3_I
* http://harinisuresh.com/2016/10/09/lstms/

### Bonus Challenges (not required!):
1. Build the forward pass of an LSTM, without using any existing RNN APIs (as above, with PyTorch or Tensorflow)
1. Build a basic RNN or LSTM in Numpy - including forward pass as well as backpropogation